In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import seaborn as sns
import pickle
import re

import csv
import networkx as nx
from operator import itemgetter

In [ ]:
disease_list = ['Cardiovascular','Dementia','Diabetes','HIV','Intestinal',
                'Kidney','Malnutrition','Maternal','Meningitis','Neonatal',
               'Respiratory','Tuberculosis','Liver']

for disease in disease_list:
    filename = disease + '/#' + disease + '.csv'
    df=pd.read_csv(filename,encoding='iso-8859-1')
    
    #select useful columns 
    network={'User name':df['screen_name'],
        'Text':df['text'],
        'Create time':df['created_at'],
        'Source':df['source'],
        'Favourite count':df['favorite_count'],
        'Retweet count':df['retweet_count'],
        'Hashtags':df['hashtags'],
        'url':df['urls_url'],
        'follower':df['followers_count'],
        'following':df['friends_count'],
        'location':df['location']}
    
    networkcsv=pd.DataFrame.from_dict(network,orient='index').transpose()
    
    # follower
    follower_csv = networkcsv[['User name','follower','location','url']]
    follower_csv = follower_csv.sort_values('follower',ascending=False)
    follower_csv.reset_index(inplace = True)
    follower_csv = follower_csv.drop(columns = 'index')
    follower_csv = follower_csv.drop_duplicates(subset='User name',keep='first',inplace=False)
    follower_csv = follower_csv.iloc[0:20]
    follower_csv.reset_index(inplace = True)
    follower_csv = follower_csv.drop(columns = 'index')
    
    follower_name = disease + '/#' + disease + 'follower.csv'
    follower_csv.to_csv(follower_name,encoding='utf-8')
    
    #network
    networktext=networkcsv['Text']
    
    pattern = re.compile(r'@(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mentioned =[]
    for name1 in networktext: # Find the user who has #
        name=re.findall(pattern,name1)
        if name!=[]:
            mentioned.append(name)
        else:
            mentioned.append(1)
            
    usernames=networkcsv['User name']
    usernamess=[]
    for i in usernames:
        usernamess.append(i)
        
    mention_net ={'username':usernamess,
         'mentioned':mentioned}
    
    mention_net = pd.DataFrame.from_dict(mention_net,orient ='index').transpose()
    
    x=0
    test=mention_net['mentioned'] # Select the user who doesn't have #
    droplist=[]
    leavelist=[]
    for tag in test:
        if tag is 1:
            droplist.append(x)
        else:
            leavelist.append(x)
        x=x+1
    
    newmentioned=mention_net.drop(droplist)
    
    te=newmentioned['username']
    lovelyname=[] 
    for i in te:
        lovelyname.append(i) # Create a new name list
        
    tag1=newmentioned['mentioned']
    tag2=[]
    newuser=[]
    x=0
    for i in tag1:
        for s in i:
            tag2.append(s) # Create a new tag list
            newuser.append(lovelyname[x]) #Create a new username with all tags
        x=x+1
        
    socialnetwork={'Username':newuser,
              'Tag':tag2}
    
    socialnetworkcsv=pd.DataFrame.from_dict(socialnetwork,orient ='index').transpose() #Create a new csv file
    
    nsocialnetworkcsv=socialnetworkcsv.set_index('Username') 
    
    mantion_name = disease + '/#' + disease + 'mention.csv'
    nsocialnetworkcsv.to_csv(mantion_name)
    
    
    with open(mantion_name, 'r') as edgecsv: # Open the file
        edgereader = csv.reader(edgecsv) # Read the csv     
        edges = [tuple(e) for e in edgereader][1:] # Retrieve the data
    
    G = nx.Graph()
    G.add_edges_from(edges)
    print(disease,nx.info(G))
    
    print(disease,nx.is_connected(G))

# Next, use nx.connected_components to get the list of components,
# then use the max() command to find the largest one:
    components = nx.connected_components(G)
    largest_component = max(components, key=len)

# Create a "subgraph" of just the largest component
# Then calculate the diameter of the subgraph, just like you did with density.

    subgraph = G.subgraph(largest_component)
    diameter = nx.diameter(subgraph)
    print(disease,"Network diameter of largest component:", diameter)
    
    layout = nx.spring_layout(G)
    plt.figure(figsize=(150,150))
    nx.draw_networkx(G, layout, with_lables=False,front_size=30,node_color="skyblue")
    plt.title('Network')
    png1_name = disease + '/#'+ disease + '1.png'
    plt.savefig(png1_name, dpi=100)
    
    layout = nx.spring_layout(subgraph)
    plt.figure(figsize=(150,150))
    nx.draw_networkx(subgraph, layout, with_lables=False,front_size=30)
    png2_name = disease + '/#' + disease + '2.png'
    plt.savefig(png2_name, dpi=100)
    
    # degree
    degree_dict = dict(G.degree(G.nodes()))

    sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)
    degree_user=[]
    degree_value = []
    for d in sorted_degree:
        #print(d)
        degree_user.append(d[0])
        degree_value.append(d[1])
    
    #betweenness
    betweenness_dict = nx.betweenness_centrality(G) # Run betweenness centrality

    sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)

    betweeness_user=[]
    betweeness_value=[]

    for b in sorted_betweenness:
        #print(b)
        betweeness_user.append(b[0])
        betweeness_value.append(b[1])
        
    Degree_information={'Degree User':degree_user,
             'Degree Value':degree_value}
    Betweenness_information = {'Betweenness User':betweeness_user,
                          'Betweenness Value':betweeness_value}
    
    de_informationcsv=pd.DataFrame.from_dict(Degree_information,orient='index').transpose()
    de_name = disease + '/#'+ disease + 'degree.csv'
    de_informationcsv.to_csv(de_name)

    Be_informationcsv=pd.DataFrame.from_dict(Betweenness_information,orient='index').transpose()
    Be_name = disease + '/#'+ disease + 'between.csv'
    Be_informationcsv.to_csv(Be_name)
    
